# Materials
* [Bunch of articles](http://www.mitpressjournals.org/doi/pdf/10.1162/neco.1993.5.6.954) - I strongly recomment this resource, cause it hosts most actual (by year of publishing) articles.
* [realization on C++](https://github.com/BelBES/ESOINN)
* [ESOINN algorithm](http://cs.nju.edu.cn/rinc/SOINN/e-soinn.pdf)
* [Detailed article](http://www.haselab.info/soinn-e.html)

In [1]:
import numpy as np
import pickle

In [302]:
class Node:
    def __init__(self, density=0, feature_vector=[0, 0]):
        self.density = density
        self.subclass_id = -1
        self.feature_vector = np.array(feature_vector, dtype=float)
    def __repr__(self):
        return str(self.density)

class Graph:
    def __init__(self):
        self.nodes = {}
        self.neighbors = {}
        self.edges = {}
    def __repr__(self):
        representation = "{\n"
        for i, node in enumerate(self.nodes):
            representation += f"{i:<3}:\t {node}\n"
        representation += "}\n"
        return representation

In [193]:
# with open(r"src/suspended_undirected_graph", "rb") as file:
#     g = pickle.load(file)
# g
# with open(r"src/suspended_undirected_graph", "wb") as file:
#     pickle.dump(g, file)

with open(r"src/suspended_undirected_graph_enchanced", "rb") as file:
    g = pickle.load(file)
# with open(r"src/suspended_undirected_graph", "wb") as file:
#     pickle.dump(g, file)

In [203]:
def find_neighbors_local_maxes(g, node_id):
    apexes = set()
    visited = {node_id}
    
    queue = []
    node_density = g.nodes[node_id].density
    for neighbor_id in g.neighbors.get(node_id, set()) - visited:
        if g.nodes[neighbor_id].density > node_density:
            queue.append(neighbor_id)
        visited.add(neighbor_id)
    
    for vertex in queue:
        is_local_max = True
        vertex_density = g.nodes[vertex].density
        for neighbor_id in g.neighbors[vertex]:
            if g.nodes[neighbor_id].density > vertex_density:
                if neighbor_id not in visited:
                    queue.append(neighbor_id)
                is_local_max = False
            visited.add(neighbor_id)
        if is_local_max:
            apexes.add(vertex)
        visited.add(vertex)
    if not apexes:
        return {node_id}
    return apexes

In [15]:
def max_apex_in_class(g, start_node_id: int):
    max_apex_id = start_node_id
    visited = {start_node_id}
    queue = list(g.neighbors.get(start_node_id, set()) - visited)
    for vertex in queue.copy():
        if g.nodes[max_apex_id].density < g.nodes[vertex].density:
            max_apex_id = vertex
        visited.add(vertex)
        queue.remove(vertex)
        queue.extend([
            node for node in g.neighbors[vertex] - visited
            if node not in visited
        ])
    return max_apex_id, visited

In [16]:
%%timeit

d = {}
for i in range(34):
    d[i+1] = find_neighbors_local_maxes(g, i+1)

1000 loops, best of 3: 339 µs per loop


In [218]:
d = {}
for i in range(34):
#     print(f'for vertex {i+1}')
    d[i+1] = find_neighbors_local_maxes(g, i+1)
#     print(f"{i+1:<6}: {find_neighbors_local_maxes(g, i+1)}")
print(d == check)

True


In [216]:
check = {1     : {1},
2     : {2},
3     : {3},
4     : {4},
5     : {5},
6     : {6},
7     : {7},
8     : {8},
9     : {9},
10    : {10},
11    : {3, 4, 5},
12    : {1, 2, 5, 6},
13    : {1, 2, 5, 6},
14    : {1, 2, 5, 6},
15    : {1, 5, 6},
16    : {1, 2, 5, 6},
17    : {1, 6},
18    : {6},
19    : {6},
20    : {6},
21    : {6},
22    : {6},
23    : {6, 7},
24    : {7},
25    : {7},
26    : {1, 2, 5, 6, 7, 8, 9},
27    : {1, 2, 6, 7, 8, 9},
28    : {1, 2, 6, 8, 9},
29    : {8, 9},
30    : {8, 9, 2, 6},
31    : {2, 6, 8, 9, 10},
32    : {9, 10},
33    : {9, 10, 2},
34    : {2}}

In [21]:
s, l = max_apex_in_class(g, 26)
print(s)
print(l)

1
{1, 7, 8, 13, 25, 26, 29, 30}


In [172]:
def create_edges(g, nodes_ids):
    for node_id in nodes_ids:
        if node_id not in g.neighbors:
            g.neighbors[node_id] = set()
        for insert_id in nodes_ids:
            if insert_id != node_id:
                g.neighbors[node_id] |= {insert_id}
                nodes_pair = (min(node_id, insert_id), max(node_id, insert_id))
                g.edges[nodes_pair] = 0

In [174]:
G = Graph()

for i in g.nodes:
    G.nodes.update({i:Node(g.nodes[i].density)})
    G.neighbors.update({i:g.neighbors[i]})
    
G.nodes[1].feature_vector = np.array([2.5, 2.5])
G.nodes[2].feature_vector = np.array([5.25, 2])
G.nodes[3].feature_vector = np.array([6, 1.5])
G.nodes[4].feature_vector = np.array([6, 0.5])
G.nodes[5].feature_vector = np.array([5.25, 1])
G.nodes[6].feature_vector = np.array([2.875, 0.5])
G.nodes[7].feature_vector = np.array([1, 3.375])
G.nodes[8].feature_vector = np.array([4.5, 3.75])
G.nodes[9].feature_vector = np.array([5.75, 4.25])
G.nodes[10].feature_vector = np.array([6.75, 3.75])
G.nodes[11].feature_vector = np.array([5.5, 1.125])
G.nodes[12].feature_vector = np.array([5.25, 1.375])
G.nodes[13].feature_vector = np.array([4.25, 1.625])
G.nodes[14].feature_vector = np.array([4.25, 1.25])
G.nodes[15].feature_vector = np.array([3.5, 0.5])
G.nodes[16].feature_vector = np.array([3, 2.25])
G.nodes[17].feature_vector = np.array([2.75, 1.75])
G.nodes[18].feature_vector = np.array([2, 1.25])
G.nodes[19].feature_vector = np.array([1.5, 1])
G.nodes[20].feature_vector = np.array([0.75, 1])
G.nodes[21].feature_vector = np.array([1, 1.5])
G.nodes[22].feature_vector = np.array([1.25, 2.5])
G.nodes[23].feature_vector = np.array([1, 2.75])
G.nodes[24].feature_vector = np.array([1.75, 3])
G.nodes[25].feature_vector = np.array([2, 3.25])
G.nodes[26].feature_vector = np.array([3, 4.25])
G.nodes[27].feature_vector = np.array([3.25, 3.625])
G.nodes[28].feature_vector = np.array([3, 3])
G.nodes[29].feature_vector = np.array([4.5, 4.25])
G.nodes[30].feature_vector = np.array([4.5, 3.375])
G.nodes[31].feature_vector = np.array([5.75, 3.25])
G.nodes[32].feature_vector = np.array([6.25, 3.25])
G.nodes[33].feature_vector = np.array([6.5, 2.5])
G.nodes[34].feature_vector = np.array([6.75, 2])

for i in G.nodes:
    for j in G.neighbors[i]:
        create_edges(G,(i,j))

G.edges

{(1, 16): 0,
 (1, 17): 0,
 (1, 26): 0,
 (1, 27): 0,
 (1, 28): 0,
 (2, 12): 0,
 (2, 13): 0,
 (2, 30): 0,
 (2, 31): 0,
 (2, 34): 0,
 (3, 11): 0,
 (4, 11): 0,
 (5, 11): 0,
 (5, 12): 0,
 (5, 13): 0,
 (5, 14): 0,
 (5, 15): 0,
 (6, 13): 0,
 (6, 14): 0,
 (6, 15): 0,
 (6, 18): 0,
 (6, 30): 0,
 (7, 23): 0,
 (7, 24): 0,
 (7, 25): 0,
 (7, 26): 0,
 (7, 27): 0,
 (8, 26): 0,
 (8, 27): 0,
 (8, 28): 0,
 (8, 29): 0,
 (8, 30): 0,
 (8, 31): 0,
 (9, 29): 0,
 (9, 30): 0,
 (9, 31): 0,
 (9, 32): 0,
 (10, 31): 0,
 (10, 32): 0,
 (12, 13): 0,
 (12, 14): 0,
 (12, 15): 0,
 (13, 14): 0,
 (13, 16): 0,
 (13, 17): 0,
 (13, 26): 0,
 (14, 15): 0,
 (14, 16): 0,
 (14, 17): 0,
 (15, 16): 0,
 (15, 17): 0,
 (17, 18): 0,
 (18, 19): 0,
 (19, 20): 0,
 (19, 21): 0,
 (20, 21): 0,
 (21, 22): 0,
 (21, 23): 0,
 (22, 23): 0,
 (23, 24): 0,
 (24, 25): 0,
 (25, 26): 0,
 (25, 27): 0,
 (26, 29): 0,
 (26, 30): 0,
 (27, 29): 0,
 (27, 30): 0,
 (28, 29): 0,
 (28, 30): 0,
 (29, 31): 0,
 (30, 31): 0,
 (32, 33): 0,
 (33, 34): 0}

In [434]:
def metrics(x, y):
    return np.sqrt(
                     np.sum(np.square(np.array(x) - np.array(y)))
                 )

def remove_edges(g, nodes_ids):
    for node_id in nodes_ids:
        for remove_id in nodes_ids:
            if remove_id != node_id and remove_id in g.neighbors[node_id]:
                g.neighbors[node_id] -= {remove_id}
                nodes_pair = (min(node_id, remove_id), max(node_id, remove_id))
                if nodes_pair in g.edges:
                    del g.edges[nodes_pair]
        if not g.neighbors[node_id]:
            del g.neighbors[node_id]

def mark_subclasses(g, node_id: int,
                    overlap_nodes: dict,
                    visited: set):
    
    visited.add(node_id)
    queue = [node_id]
#   Set for keepping node_ids which need to remove edges
    remove_set = set()
    
#   Mark apex
    g.nodes[node_id].subclass_id = node_id
    apex_id = g.nodes[node_id].subclass_id


    for vertex in queue:
        g.nodes[vertex].subclass_id = apex_id
        vertex_density = g.nodes[vertex].density
        visited.add(vertex)

        for neighbor_id in g.neighbors[vertex].copy():
            if g.nodes[neighbor_id].density < vertex_density:
#               Если мы нашли нейрон с плотностью меньше, чем у родителя
#               То нужно понять стоит ли нам его маркировать, либо маркоровать будет кто-то другой
                min_dist = calc_neighbor_min_dist(g,neighbor_id)
                if  metrics(g.nodes[vertex].feature_vector, g.nodes[neighbor_id].feature_vector) <= min_dist:
                    # Если мы ранее не посещяли узел, то значит совсем что-то новое
                    if neighbor_id not in visited:
                        queue.append(neighbor_id)
#               Если маркировать будет кто-то другой, значит это overlap area
                else:
                    overlap_nodes.update({neighbor_id: min_dist})
                    remove_set.add(neighbor_id)

#   Удаление ребер 
    for remove_id in remove_set:
#       Проверка на то, что к данному узлу из вершины никто НЕ смог пробиться
        if g.nodes[remove_id].subclass_id != apex_id:
            for neighbor_id in g.neighbors[remove_id].copy():
                if g.nodes[neighbor_id].subclass_id == apex_id:
                    remove_edges(g, (remove_id, neighbor_id))
                    
    return overlap_nodes, visited


def calc_neighbor_min_dist(g, node_id: int) -> float:
    min_dist = float('inf')
    for neighbor_id in g.neighbors[node_id]:
        if g.nodes[neighbor_id].density > g.nodes[node_id].density:
            dist = metrics(g.nodes[neighbor_id].feature_vector,
                                g.nodes[node_id].feature_vector)
            if min_dist > dist:
                min_dist = dist

    return min_dist

# @TODO: subclasses remove connections between subclasses
# algorithm 3.1
def separate_subclasses(g):
    visited_in_mark = set()
    overlap_nodes = dict()
    for node_id in g.nodes:
        if node_id not in visited_in_mark:
            apexes = find_neighbors_local_maxes(g,node_id)
            for apex in apexes:
                overlap_nodes, visited_in_mark = mark_subclasses(g, apex,
                                         overlap_nodes,
                                         visited_in_mark)


In [435]:
with open(r"src/suspended_undirected_graph_enchanced", "rb") as file:
    g = pickle.load(file)

separate_subclasses(g)

In [436]:
for i in g.nodes:
    print(f'{i}: {g.nodes[i].subclass_id}')
    
for i in g.nodes:
    print(f'{i}: {g.neighbors.get(i, set())}')

g.edges

1: 1
2: 2
3: 3
4: 4
5: 5
6: 6
7: 7
8: 8
9: 9
10: 10
11: 5
12: 5
13: 2
14: 2
15: 6
16: 1
17: 1
18: 6
19: 6
20: 6
21: 6
22: 6
23: 6
24: 7
25: 7
26: 7
27: 8
28: 1
29: 8
30: 8
31: 9
32: 10
33: 2
34: 2
1: {16, 17, 28}
2: {34, 13}
3: set()
4: set()
5: {11, 12}
6: {15, 18}
7: {24, 25, 26}
8: {27, 29, 30}
9: {31}
10: {32}
11: {5}
12: {5}
13: {2, 14}
14: {13}
15: {6}
16: {1}
17: {1}
18: {19, 6}
19: {18, 20, 21}
20: {19, 21}
21: {19, 20, 22, 23}
22: {21, 23}
23: {21, 22}
24: {25, 7}
25: {24, 26, 7}
26: {7, 25}
27: {8, 29, 30}
28: {1}
29: {8, 27}
30: {8, 27}
31: {9}
32: {10}
33: {34}
34: {33, 2}


{(1, 16): 0,
 (1, 17): 0,
 (1, 28): 0,
 (2, 13): 0,
 (2, 34): 0,
 (5, 11): 0,
 (5, 12): 0,
 (6, 15): 0,
 (6, 18): 0,
 (7, 24): 0,
 (7, 25): 0,
 (7, 26): 0,
 (8, 27): 0,
 (8, 29): 0,
 (8, 30): 0,
 (9, 31): 0,
 (10, 32): 0,
 (13, 14): 0,
 (18, 19): 0,
 (19, 20): 0,
 (19, 21): 0,
 (20, 21): 0,
 (21, 22): 0,
 (21, 23): 0,
 (22, 23): 0,
 (24, 25): 0,
 (25, 26): 0,
 (27, 29): 0,
 (27, 30): 0,
 (33, 34): 0}